In [1]:
import os
from glob import glob
import warnings
import random
warnings.filterwarnings('ignore')
import datetime

import import_ipynb
import abuse_detecting_preprocessing as pp
import abuse_detecting_databuild as b

import pandas as pd
import numpy as np

import fasttext

import tensorflow as tf

_1DCNN_model = tf.keras.models.load_model('1DCNN_model.h5')
BiLSTM_model = tf.keras.models.load_model('BiLSTM_model.h5')
GRU_model = tf.keras.models.load_model('GRU_model.h5')
fasttext_model = fasttext.load_model('/Users/kim-yongjun/Documents/abuse_detecting/fasttext_model.bin')


importing Jupyter notebook from abuse_detecting_preprocessing.ipynb
['ㅇㅡㄴㄹㅡㄹㅈㅏㄹㅇㅣ-ㄱㅣㄴㄷㅏ-ㅎㅏ-', 'ㄴㅡㄴㄹㅗㄹㅇㅢ-ㅅㅣㄴㅇㅣ-ㄷㅏ-'] 

['t1ㅇㅡㄴlckㄹㅡㄹㅈㅏㄹㅇㅣ-ㄱㅣㄴㄷㅏ-ㅎㅏ-', 'fakerㄴㅡㄴㄹㅗㄹㅇㅢ-ㅅㅣㄴㅇㅣ-ㄷㅏ-'] 

importing Jupyter notebook from abuse_detecting_databuild.ipynb


In [2]:
def erase_bot(data):

    # 첫 번째 열의 값이 '@'로 시작하는 행을 필터링하여 봇으로 간주하고 제거
    idxs = data[data.str.startswith('@') == True].index
    data.loc[idxs] = None
    bot_cnt = len(idxs)
    print('data.shape : ', data.shape, 'None 처리한 봇 수 : ', bot_cnt, '\n')
    return data
# erase_bot 함수 사용 예시
# data = erase_bot(data)


In [3]:
def null_finder(data): # 공백만 있는 메세지 제거
    print('공백 제거 전\n', data.isnull().value_counts())
    data = data.dropna()
    print('공백 제거 후 \n', data.isnull().value_counts())
    
    return data

In [4]:
def erase_emoticons(data):
    # 각 문장에 대해 이모티콘을 제거하는 작업을 수행
    cleaned_data = []
    for sentence in data:
        while '::' in sentence:
            start = sentence.find('::')
            end = sentence.find('::', start + 2)
            if end == -1:
                break
            sentence = sentence[:start] + sentence[end + 2:]
        cleaned_data.append(sentence)
    return cleaned_data


In [10]:
def soft_ensemble(models, data, threshold): 
# 욕설 0 , 정상 1 / 세 모델 결과의 평균이 0.3 미만일 경우(확실한 경우)를 0 으로 라벨링
# Pseudo labeled data를 만들기 위함.
    pred = np.column_stack([m.predict(data) for m in models])
    pred = np.mean(pred, axis = 1)
    pred = np.array([0 if p <= threshold else 1 for p in pred])
    return pred
    

In [19]:
def makesample(size1, size2):
# 전체 데이터에서 size만큼의 크기의 데이터 추출
    data = pd.DataFrame()
    r = glob(os.getcwd() + '/chatting_datas/*.csv')
    for i in r:
        data = pd.concat([data, pd.read_csv(i)], ignore_index = True)
        
    df = data['message']
    message = erase_bot(df)
    message = null_finder(message)
    message = pd.Series(erase_emoticons(message))
    message = message.sample(size1)
    
    vectors = b.making_x_train(message, embedding_length, fasttext_model, n_gram)
    pred = soft_ensemble(models, vectors, threshold)
    message = pd.DataFrame(message)
    message['label'] = pred
    message = message.rename(columns = {0 : 'message' })
    
    return message.sample(size2)

In [20]:
def activation_learning_data(models, size):
# 세 모델의 결과값 평균치가 0.3 이상 0.7 미만인 데이터들만 추출 (욕설인지 아닌지 애매한 데이터들)
    m, vec = makesample(size)
    pred = np.column_stack([m.predict(vec) for m in models])
    pred = np.mean(pred, axis = 1)
    pred = np.array([True if 0.3 <= p <= 0.7 else False for p in pred])
    m = pd.DataFrame(m)
    m['label'] = pred
    target = m[m['label'] == True]
    return target[['message']]

In [23]:
models = [_1DCNN_model, BiLSTM_model, GRU_model]
embedding_length = 40
n_gram = 5
size1 = 50000
size2 = 10000
threshold = 0.2

today = datetime.datetime.today().strftime("%Y_%m_%d_%H_%M") 
messages = makesample(size1, size2)
messages = pd.DataFrame(messages)
messages

data.shape :  (1008951,) None 처리한 봇 수 :  9367 

공백 제거 전
 message
False    999566
True       9385
Name: count, dtype: int64
공백 제거 후 
 message
False    999566
Name: count, dtype: int64
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step


,message,label
838636,400원 모아서 서폿 탬 사오는거잖아.. 롤알못들아,1
396273,너무 잔인해요…,1
813862,개상혁 vs 대상혁,1
748665,ㅋㅋㅋㅋㅋ,1
677780,ㅋㅋㅋㅋ...,1
...,...,...
710167,ㅈㄱㅊㅇ,1
337389,세나 cs 얘기하는애들은 진짜 나가라 뭐하는 억까들이냐 ㅋㅋ,1
976360,이거 진짜 킹겐이 정글 다풀어줌,1
742533,우디르아 라인전 지면 게임많이 힘든데,1


In [24]:
messages.to_csv(os.getcwd() + f'/chatting_datas/labeled_datas/{today}_pseudo.csv')
print(f'{size2}개의 수도 레이블 생성 완료.')

10000개의 수도 레이블 생성 완료.


In [10]:
act_data = activation_learning_data(models, 10000)
act_data.to_csv(os.getcwd() + f'/chatting_datas/{today}_act_learning.csv')

data.shape :  (1008951,) None 처리한 봇 수 :  9367 

공백 제거 전
 message
False    999566
True       9385
Name: count, dtype: int64
공백 제거 후 
 message
False    999566
Name: count, dtype: int64
An error occurred: 63
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
